<a href="https://colab.research.google.com/github/skovz99/Cell_Cluster_Morphology/blob/main/Cell_Cluster_Morphology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Install UMAP***

In [ ]:
pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 993.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82807 sha256=103f68be8ecbf64c76140de5d89b33cee64c39145f2c01e823267781e753f99c
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=f8063552971108afd59022dee0f8e1b7b35c9461f0b1c4942057f185b1df7d9d
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


### ***Run Functions Necessary for Cell Cluster Morphology Analysis***

This code currently only supports finding 3 nearest neighbors. Functions will have to be updated to allow for any desired number

In [ ]:
# purpose of this code is to allow for the determination of the shape that clusters of nearest neighboring cells take from image data using only the coordinates of the NN

# define a function to find the nearest neighbors (k) to all centroid coordinates of objects identified in the image
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
import math
from sklearn.metrics import r2_score
from scipy.stats import linregress
import os
import skimage
import skimage.io
import skimage.util as util
from skimage import filters, measure
from scipy.stats import percentileofscore
import warnings
warnings.filterwarnings("ignore")

# extract the centroid coordinates of all identified objects within the image and save as an excel file
def image_analysis(image, thresholding = False, threshold_int = 50):
  centroidx1 = []
  centroidy1 = []
  if thresholding == 'True':
    image[image <= threshold_int] = 0
  normalizationA = (image.shape[0]*image.shape[1])/(4*3.04)
  image_16bit = skimage.util.img_as_uint(image)
  threshold_value = filters.threshold_otsu(image_16bit)
  binary_image = image_16bit > threshold_value
  labeled_image = measure.label(binary_image)
  region_properties = measure.regionprops(labeled_image)
  for region in region_properties:
    centroidx1.append(region.centroid[1])
    centroidy1.append(region.centroid[0])
  a = pd.DataFrame()
  a['X'] = centroidx1
  a['Y'] = centroidy1
  a = a.dropna()
  a.to_excel('image.xlsx')

data = pd.read_excel('image.xlsx')

def nearest_neighbors(k, X_coord, Y_coord, df): # k is the number of nearest neighbors to consider when forming the cluster (k is essentially cluster size)
  zipped = [list(i) for i in zip(X_coord, Y_coord)]
  nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(zipped)
  distances, indices = nbrs.kneighbors(zipped)
  nn_data = []
  for i in range(len(indices)):
    x_coord = X_coord[i]
    y_coord = Y_coord[i]
    nn_list = [(X_coord[idx], Y_coord[idx]) for idx in indices[i]]
    nn_data.append([x_coord, y_coord] + [xy for neighbor in nn_list for xy in neighbor])
  columns = ['X', 'Y']
  for i in range(k):
    columns.extend([f'NN {i+1} X', f'NN {i+1} Y'])
  indices_df = pd.DataFrame(nn_data, columns=columns)
  indices_df = indices_df.drop(['X', 'Y'], axis=1)
  return indices_df

NN = nearest_neighbors(k=3, X_coord = data_df['X'], Y_coord = data_df['Y'], df = data_df)

# define a function to calculate the distance between each point and every other point in the row
def cluster_distances(nearest_neighbors_df):
  # compare first X,Y to second X,Y distances
  nearest_neighbors_df['First Distance'] = np.sqrt(((nearest_neighbors_df['NN 1 X'] - nearest_neighbors_df['NN 2 X']) ** 2) + ((nearest_neighbors_df['NN 1 Y'] - nearest_neighbors_df['NN 2 Y'])**2))
  nearest_neighbors_df['Second Distance'] = np.sqrt(((nearest_neighbors_df['NN 1 X'] - nearest_neighbors_df['NN 3 X']) ** 2) + ((nearest_neighbors_df['NN 1 Y'] - nearest_neighbors_df['NN 3 Y'])**2))
  nearest_neighbors_df['Third Distance'] = np.sqrt(((nearest_neighbors_df['NN 2 X'] - nearest_neighbors_df['NN 3 X']) ** 2) + ((nearest_neighbors_df['NN 2 Y'] - nearest_neighbors_df['NN 3 Y'])**2))
  return nearest_neighbors_df

NN_a = cluster_distances(NN)

# define a function to calculate the angle of the slope relative to the x axis of the two points in the cluster furthest away
def furthest_neighbors_slope_angle(cluster_nearest_neighbors):
  remove_NNs = pd.DataFrame()
  remove_NNs['First'] = cluster_nearest_neighbors['First Distance']
  remove_NNs['Second'] = cluster_nearest_neighbors['Second Distance']
  remove_NNs['Third'] = cluster_nearest_neighbors['Third Distance']
  remove_NNs['Max'] = remove_NNs.max(axis=1)
  cluster_nearest_neighbors['Max'] = remove_NNs['Max']
  for idx, row in cluster_nearest_neighbors.iterrows():
    if row['Max'] == row['First Distance']:
      cluster_nearest_neighbors.loc[idx, ['Slope X1', 'Slope X2', 'Slope Y1', 'Slope Y2']] = [row['NN 1 X'], row['NN 2 X'], row['NN 1 Y'], row['NN 2 Y']]
    elif row['Max'] == row['Second Distance']:
      cluster_nearest_neighbors.loc[idx, ['Slope X1', 'Slope X2', 'Slope Y1', 'Slope Y2']] = [row['NN 1 X'], row['NN 3 X'], row['NN 1 Y'], row['NN 3 Y']]
    elif row['Max'] == row['Third Distance']:
      cluster_nearest_neighbors.loc[idx, ['Slope X1', 'Slope X2', 'Slope Y1', 'Slope Y2']] = [row['NN 2 X'], row['NN 3 X'], row['NN 2 Y'], row['NN 3 Y']]
  angle_to_x = np.rad2deg(np.arctan2(cluster_nearest_neighbors['Slope Y2'] - cluster_nearest_neighbors['Slope Y1'], cluster_nearest_neighbors['Slope X2'] - cluster_nearest_neighbors['Slope X1']))
  cluster_nearest_neighbors['Furthest Angle to X'] = angle_to_x/180 # this value has normalized all angles to be between -1 and 1
  slope_of_furthest = (cluster_nearest_neighbors['Slope Y2'] - cluster_nearest_neighbors['Slope Y1'])/(cluster_nearest_neighbors['Slope X2'] - cluster_nearest_neighbors['Slope X1'])
  cluster_nearest_neighbors['Slope of Furthest'] = slope_of_furthest
  cluster_nearest_neighbors['y-intercept furthest'] = (cluster_nearest_neighbors['Slope Y1']) - (cluster_nearest_neighbors['Slope of Furthest'] * cluster_nearest_neighbors['Slope X1'])
  return cluster_nearest_neighbors

angle = furthest_neighbors_slope_angle(NN_a)

# define a function to calculate the distance of every point in the cluster from the line of best fit from furthest neighbors, R squared and angle of the slope
def furthest_neighbors_perpendicular(k, sloped_nearest_neighbors):
  sloped_nearest_neighbors['Inverse Slope'] = -1 / (sloped_nearest_neighbors['Slope of Furthest'])
  for x in range(k):
    find_b_y = (sloped_nearest_neighbors[f'NN {x + 1} Y']) - (sloped_nearest_neighbors['Inverse Slope'] * sloped_nearest_neighbors[f'NN {x + 1} X'])
    sloped_nearest_neighbors[f'y-int for Inverse NN {x+1}'] = find_b_y
    find_x_int = (sloped_nearest_neighbors[f'y-int for Inverse NN {x+1}'] - sloped_nearest_neighbors[f'y-intercept furthest']) / (sloped_nearest_neighbors[f'Inverse Slope'] - sloped_nearest_neighbors[f'Slope of Furthest'])
    sloped_nearest_neighbors[f'x-int for Inverse NN {x+1}'] = find_x_int
    find_distance = np.sqrt(((sloped_nearest_neighbors[f'y-int for Inverse NN {x+1}'] - sloped_nearest_neighbors[f'NN {x+1} Y']) ** 2) + ((sloped_nearest_neighbors[f'x-int for Inverse NN {x+1}'] - sloped_nearest_neighbors[f'NN {x+1} X']) ** 2))
    sloped_nearest_neighbors[f'Distances {x+1}'] = 1 / find_distance # distance inversed so that all distances are between 0 and 1 with distances closer to 1 being points closer to the line
  # calculation of y_pred for R squared
    y_pred = ((sloped_nearest_neighbors[f'NN {x+1} X'] * sloped_nearest_neighbors['Slope of Furthest']) + (sloped_nearest_neighbors['y-intercept furthest']))
    sloped_nearest_neighbors[f'y-pred for NN {x+1} Y'] = y_pred
  # calculation of the R squared value
  r_squared_list = []
  for xx in range(len(sloped_nearest_neighbors)):
    y_pred_list = [
      sloped_nearest_neighbors.at[xx, 'y-pred for NN 1 Y'],
      sloped_nearest_neighbors.at[xx, 'y-pred for NN 2 Y'],
      sloped_nearest_neighbors.at[xx, 'y-pred for NN 3 Y']]
    y_true_list = [
      sloped_nearest_neighbors.at[xx, 'NN 1 Y'],
      sloped_nearest_neighbors.at[xx, 'NN 2 Y'],
      sloped_nearest_neighbors.at[xx, 'NN 3 Y']]
    r_squared_list.append(r2_score(y_true_list, y_pred_list))
  sloped_nearest_neighbors['R-Squared for Furthest'] = r_squared_list
  return sloped_nearest_neighbors

furthest = furthest_neighbors_perpendicular(3, angle)

# define a function to calculate the distance of every point in the cluster from the line of best fit from all neighbors, R squared and angle of the slope
def all_neighbors_sloped_angle(k, cluster_nearest_neighbors):
  remove_all = pd.DataFrame()
  for x in range(k):
    remove_all[f'NN {x+1} X'] = cluster_nearest_neighbors[f'NN {x+1} X']
    remove_all[f'NN {x+1} Y'] = cluster_nearest_neighbors[f'NN {x+1} Y']
  # create a list of all the X,Y points in the columns
  slopes = []
  angles = []
  y_intercept = []
  for idx, row in remove_all.iterrows():
    X_row = [row[f'NN {x+1} X'] for x in range(k)]
    Y_row = [row[f'NN {x+1} Y'] for x in range(k)]
    line = linregress(X_row, Y_row)
    slopes.append(line.slope)
    angle_to_x = np.rad2deg(np.arctan2(np.sum(Y_row), np.sum(X_row)))
    angles.append(angle_to_x)
    y_intercept.append(line.intercept)
  remove_all['Slope of All Neighbors'] = slopes
  remove_all['Slope Angle of All'] = [angle / 180 for angle in angles]
  remove_all['Y-intercept for All'] = y_intercept
  return remove_all

new = all_neighbors_sloped_angle(3, angle)

# define a function to calculate the distance of every point in the cluster from the line of best fit from all neighbors, R squared and angle of the slope
def all_neighbors_perpendicular(k, sloped_for_all):
  sloped_for_all['Inverse Slope'] = -1 / (sloped_for_all['Slope of All Neighbors'])
  for x in range(k):
    find_b_y = (sloped_for_all[f'NN {x + 1} Y']) - (sloped_for_all['Inverse Slope'] * sloped_for_all[f'NN {x + 1} X'])
    sloped_for_all[f'y-int for Inverse NN {x+1}'] = find_b_y
    find_x_int = (sloped_for_all[f'y-int for Inverse NN {x+1}'] - sloped_for_all[f'Y-intercept for All']) / (sloped_for_all[f'Inverse Slope'] - sloped_for_all[f'Slope of All Neighbors'])
    sloped_for_all[f'x-int for Inverse NN {x+1}'] = find_x_int
    find_distance = np.sqrt(((sloped_for_all[f'y-int for Inverse NN {x+1}'] - sloped_for_all[f'NN {x+1} Y']) ** 2) + ((sloped_for_all[f'x-int for Inverse NN {x+1}'] - sloped_for_all[f'NN {x+1} X']) ** 2))
    sloped_for_all[f'All Distances {x+1}'] = 1 / find_distance
    # calculation of y_pred for R squared
    y_pred = ((sloped_for_all[f'NN {x+1} X'] * sloped_for_all['Slope of All Neighbors']) + (sloped_for_all['Y-intercept for All']))
    sloped_for_all[f'y-pred for NN {x+1} Y'] = y_pred
  r_squared_list = []
  for xx in range(len(sloped_for_all)):
    y_pred_list = [
        sloped_for_all.at[xx, 'y-pred for NN 1 Y'],
        sloped_for_all.at[xx, 'y-pred for NN 2 Y'],
        sloped_for_all.at[xx, 'y-pred for NN 3 Y']]
    y_true_list = [
        sloped_for_all.at[xx, 'NN 1 Y'],
        sloped_for_all.at[xx, 'NN 2 Y'],
        sloped_for_all.at[xx, 'NN 3 Y']]
    r_squared_list.append(r2_score(y_true_list, y_pred_list))
  sloped_for_all['R-Squared for All'] = r_squared_list
  return sloped_for_all

all = all_neighbors_perpendicular(3, new)

# define a function to compile the import values for UMAP
def UMAP_values(k, sloped_for_all, sloped_nearest_neighbors):
  UMAP_df = pd.DataFrame()
  UMAP_df['Slope of All Neighbors'] = sloped_for_all['Slope of All Neighbors']
  UMAP_df['Slope Angle of All'] = sloped_for_all['Slope Angle of All']
  UMAP_df['R-Squared for All'] = sloped_for_all['R-Squared for All']
  avg_all_distance = []
  for idx, row in sloped_for_all.iterrows():
    all_dist = [row[f'All Distances {x+1}'] for x in range(k)]
    avg_all = (sum(all_dist))/(len(all_dist))
    avg_all_distance.append(avg_all)
  UMAP_df['Average All Distances'] = avg_all_distance
  UMAP_df['Slope of Furthest Neighbors'] = sloped_nearest_neighbors['Slope of Furthest']
  UMAP_df['Slope Angle of Furthest'] = sloped_nearest_neighbors['Furthest Angle to X']
  UMAP_df['R-Squared for Furthest'] = sloped_nearest_neighbors['R-Squared for Furthest']
  avg_furthest_distance = []
  for idx, row in sloped_nearest_neighbors.iterrows():
    furthest_dist = [row[f'Distances {x+1}'] for x in range(k)]
    avg_furthest = (sum(furthest_dist))/(len(furthest_dist))
    avg_furthest_distance.append(avg_furthest)
  UMAP_df['Average Furthest Distances'] = avg_furthest_distance
  # computing the difference between slope, slope angle, r-squared between furthest and all categories
  UMAP_df['Slope Difference'] = np.sqrt((UMAP_df['Slope of All Neighbors'] - UMAP_df['Slope of Furthest Neighbors']) ** 2)
  UMAP_df['Slope Angle Difference'] = np.sqrt((UMAP_df['Slope Angle of All'] - UMAP_df['Slope Angle of Furthest']) ** 2)
  UMAP_df['R-Squared Difference'] = np.sqrt((UMAP_df['R-Squared for All'] - UMAP_df['R-Squared for Furthest']) ** 2)
  return UMAP_df

UMAP_val = UMAP_values(3, all, furthest)


In [ ]:
# define a function to combine all the excel sheets for UMAP plotting
def combine_excel(*excel_sheets):
  combined_data = pd.DataFrame()
  for excel_sheet in excel_sheets:
    sheet_data = pd.read_excel(excel_sheet)
    combined_data = pd.concat([combined_data, sheet_data], axis=0, ignore_index=True)
  return combined_data

# example combination
excel_file1 = "path/to/excel/file1.xlsx"
excel_file2 = "path/to/excel/file2.xlsx"
combined_df = combine_excel(excel_file1, excel_file2)


# define a function to plot the UMAP values for distances (x2), slope difference, slope angle difference and R-squared difference
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
def UMAP_Plot(values_UMAP):
  # make sure the number of UMAP neighbors is less than the size of the dataset
  parameters = umap.UMAP(n_neighbors = 15, min_dist = 0.1) # n_neighbors = 15, min_dist=0.1 --> if I did not include this code, then the default parameters are 15 NN and 0.1 min dist. I should play around with this as higher neighbors will help with big picture (small values are for local detail) and high distance will make data points cluster loosely together (small is for tight clusters)
  UMAP_values = values_UMAP[["Average All Distances","Average Furthest Distances", "Slope Difference", "Slope Angle Difference", "R-Squared Difference"]].values
  scaled_UMAPprac_data = StandardScaler().fit_transform(UMAP_values)
  embedding = parameters.fit_transform(scaled_UMAPprac_data)
  embedding.shape
  plt.scatter(embedding[:, 0],embedding[:, 1])
  plt.gca().set_aspect('equal', 'datalim')
  plt.xlabel('UMAP-1')
  plt.ylabel('UMAP-2')

plot = UMAP_Plot(combined_df)
plot

## ***Example Usage***

In [ ]:
### load the Images ###
image = skimage.io.imread('/content/AIM 2 D90-C5-DAPI-2.tif')

# extract centroid coordinates from the image
image_analysis(image, thresholding = False, threshold_int = 50)
data = pd.read_excel('image.xlsx')

# find nearest neighboring cells
NN = nearest_neighbors(k=3, X_coord = data['X'], Y_coord = data['Y'], df = data)

# record the distances between the nearest neighbors
NN_a = cluster_distances(NN)

angle = furthest_neighbors_slope_angle(NN_a)
furthest = furthest_neighbors_perpendicular(3, angle)
new = all_neighbors_sloped_angle(3, angle)
all = all_neighbors_perpendicular(3, new)
UMAP_val = UMAP_values(3, all, furthest)

# save the UMAP_val for each image as an excel sheet that can later be combined for the UMAP_Plot
UMAP_val.to_excel('UMAP_val.xlsx')

# example combination
excel_file1 = "path/to/excel/file1.xlsx"
excel_file2 = "path/to/excel/file2.xlsx"
combined_df = combine_excel(excel_file1, excel_file2)

plot = UMAP_Plot(combined_df)